### Main Function 1: Codestitch to Svelte
#### Main function: codestitch_to_svelte

#### Helper Functions
- define_filenames: creates a string filename for the files to read and write
- read_file: 
    - Input: filename string 
    - Output: content of the file string 

### Main Function 2: Codestitch to JSON 
#### Main function: 

#### Helper Functions


### Main Function 3: Move CSS to relevant file
#### Main function: 

#### Helper Functions


In [221]:
"""
Helper Functions
"""
###############################################################
# Helper functions for Main Function 1: component_to_svelte
###############################################################
# Write the filename variables 
def define_filenames(stitch_name, svelte_file):
    
    # Read files
    html_filename = f"{stitch_name}/{stitch_name}.html"
    
    # Write files
    svelte_filename = f"{stitch_name}/{stitch_name}_modified/{svelte_file}.svelte"
    
    return html_filename, svelte_filename

def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()
    return content

def write_files(filename, content):
    with open(filename, "w") as f:
        f.write(content)

# Turns a 2D list into a 1D list
def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:        
            yield item

# Extracts the string text between the curly brackets {string}            
def extract_between_brackets(s):
    if s.startswith("{") and s.endswith("}"):
        return s[1:-1]

# Takes a list and then puts each element into a string with a comma 
# between each element element1, element2, element3 etc. 
def twoD_list_to_string(variable_str, variables_list):
    flatten_var_list = flatten(variables_list)
    i = 0
    for s in flatten_var_list:
        if i == 0:
            variable_str += s
        else: 
            variable_str = f"{variable_str}, {s}"
        i += 1
    return variable_str

# Writes the initial hard-coded content at the start of the svelte file
def write_hardcoded_content(variable_str, stitch_name, soup):
    script_tag = f"<script>\n    export let {variable_str};\n</script>"
    comment = f"<!-- ============================================ -->\n<!--                {stitch_name}                  -->\n<!-- ============================================ -->"
    modified_html_content = str(soup)
    svelte_file_content = f"""{script_tag}\n\n{comment}\n\n{modified_html_content}"""
    return svelte_file_content
################################################################################################################
################################################################################################################
# Delete extract_tag once I don't need it anymore 
################################################################################################################
# Helper function written for the <label> tag because it has nested tags
# The nested tags get deleted with the current modification method
def extract_tag(label):
    # extract --> delete --> replace --> insert
    extract_tag_all = label.find_all()
    print(extract_tag_all)
    extracted_tag = ""
    for extract in extract_tag_all:
        extract_name = extract.name
        if extract_name == "input":
            extracted_tag = label.input.extract()
        elif extract_name == "textarea":
            extracted_tag = label.textarea.extract()
        elif extract_name == "select":
            extracted_tag = extract
        else: 
            pass
    return label, extracted_tag

################################################################################################################
def modify_nested(soup, variables_list, content_list, current_tag):
    # extract --> delete --> replace --> insert
    # Parent tag 
    all_tag = soup.find_all(current_tag)
    print(f"all_tag: {all_tag}")
    i = 0
    for tag in all_tag: 
        print("Entered")
        # If there is a string it's going to be the first element 
        content_list.append(tag.contents[0])
        json_var = "{" + {current_tag} + "." + {current_tag} + str(i) + "}"
        variables_list.append(json_var)
        # Children tags within the tag
        extracted_tag = tag.find_all()
        print(f"tag: {tag}, json_var: {json_var}, print: {extracted_tag}")
        tag.string = json_var
        tag.append(extracted_tag)
        i += 1
    
    return soup, variables_list, content_list

"""
def modify_label_tags(self,soup, tag_list):
    all_label= soup.find_all('label')
    i = 0
    variables_list = []
    content_list = []

    for label in all_label:
        content_list.append(label.contents[0])
        label_var = "{label.label" + str(i) + "}"

        variables_list.append(label_var)
        label, extracted_tag = extract_tag(label)

        label.string = label_var
        label.append(extracted_tag)
        i += 1

    self.content_list_master.append(content_list)
    self.variables_list_master.append(variables_list)
    tag_list.append('label')
    return tag_list
"""
################################################################################################################
################################################################################################################

# First it appends the "string content" to a list
# Then it creates the variable name to be used in the svelte file in brackets {h2.h2_0} 
# The it replaces the svelte file with the new variable
# Add the variable to a list to be used for making the JSON file later
# Now it's ready to go onto the next element so increment counter
# Return Counter and the other two lists
def modify_string(content_list, tag, i, current_tag, variables_list):
    content_list.append(current_tag.string)
    json_var = "{" + str(tag) + "_"+ str(i) + "}"
    current_tag.string = json_var
    variables_list.append(json_var)
    i += 1
    return content_list, variables_list, i
################################################################################################################
################################################################################################################
def modify_tags(tag, tag_i, attributes_list, str_bool_list, soup):
    attributes = attributes_list[tag_i]
    str_bool = str_bool_list[tag_i]
    
    all_current_tag = soup.find_all(tag)
    i = 0
    content_list = []
    variable_list = []
    for current_tag in all_current_tag: 
        print(f"tag: {tag}\n attributes_list[tag_i]: {attributes} \n str_bool: {str_bool}\n tag_i: {tag_i} \n")
        print(f"len(current_tag): {len(current_tag)}\n\n")
        if len(current_tag) <= 1:
            if str_bool == True: 
                # Keep track of original content
                content_list.append(current_tag.string)

                json_var = "{" + str(tag) + "." + str(tag) + "_"+ str(i) + "}"
                current_tag.string = json_var
                variable_list.append(json_var)

            if attributes != None: 
                j = 0
                attr_list = []
                cont_list = []
                for a in attributes:
                    cont_list.append(current_tag[a])

                    json_var = "{" + str(tag) + "." + str(a) + "_"+ str(i) + "attribute"+ "}"
                    current_tag[a] = json_var
                    attr_list.append(json_var)
                    j += 1
                content_list.append(cont_list)
                variable_list.append(attr_list)
        else: 
            print("length longer than 1")
            if str_bool == True:
                print("ENTEREEDDD")
                # If there is a string it's going to be the first element 
                content_list.append(current_tag.contents[0])
                print(f"type: {type(tag)}")
                json_var = "{" + tag + "." + tag + str(i) + "}"
                print(f"i: {i}, json_var: {json_var}")
                print(f"current_tag: {current_tag}")

                # Extract 
                extracted_tag = current_tag.find_all()
                print(f"extracted_tag: {extracted_tag}")

                # Replace 
                current_tag.string = json_var
                print(f"new current_tag: {current_tag}, {type(current_tag)}")
                h2_tag = current_tag.h2
                print(f"h2_tag: {h2_tag}")
                # Insert 
                current_tag.find_previous(tag)
                current_tag.h2.append(extracted_tag)

                variable_list.append(json_var)
                    
            # Not entirely sure how to apply the extract -> replace -> insert  
            if attributes != None:
                j = 0
                attr_list = []
                cont_list = []

                for a in attributes:
                    cont_list.append(current_tag[a])

                    json_var = "{" + str(tag) + "." + str(a) + "_"+ str(i) + "attribute"+ "}"
                    current_tag[a] = json_var
                    attr_list.append(json_var)
                    j += 1
                content_list.append(cont_list)
                variable_list.append(attr_list)
            
        i += 1
    tag_i += 1
    return tag_i, content_list,  variable_list, soup
"""
        def modify_nested_tags(self, tag, tag_i, attributes_list, str_bool_list, soup):
            # extract --> delete --> replace --> insert
            attributes = attributes_list[tag_i]
            str_bool = str_bool_list[tag_i]
            
            all_current_tag = soup.find_all(tag)
            i = 0
            variables_list = []
            content_list = []
            
            for current_tag in all_current_tag: 
                print(f"tag: {tag}\n attributes_list[tag_i]: {attributes} \n str_bool: {str_bool}\n tag_i: {tag_i} \n\n")
                if str_bool == True:
                    # If there is a string it's going to be the first element 
                    content_list.append(current_tag.contents[0])
                    print(f"type: {type(tag)}")
                    json_var = "{" + tag + "." + tag + str(i) + "}"
                    print(f"i: {i}, json_var: {json_var}")
                    print(f"current_tag: {current_tag}")
                    
                    # Extract 
                    extracted_tag = current_tag.find_all()
                    print(f"extracted_tag: {extracted_tag}")
                    
                    # Replace 
                    current_tag.string = json_var
                    
                    # Insert 
                    current_tag.append(extracted_tag)
                    
                    variable_list.append(json_var)
                    
                # Not entirely sure how to apply the extract -> replace -> insert  
                if attributes != None:
                    j = 0
                    attr_list = []
                    cont_list = []
                    
                    for a in attributes:
                        cont_list.append(current_tag[a])

                        json_var = "{" + str(tag) + "." + str(a) + "_"+ str(i) + "attribute"+ "}"
                        current_tag[a] = json_var
                        attr_list.append(json_var)
                        j += 1
                    content_list.append(cont_list)
                    variable_list.append(attr_list)
                i += 1
                
            tag_i += 1
            self.content_list_master.append(content_list)
            self.variables_list_master.append(variables_list)
            return tag_i, content_list, variable_list, soup  
"""
################################################################################################################
################################################################################################################
# Span toags modification is a little different since I'm hardcoding it a bit
def modify_span(span, json_var_name, i, content_list, variables_list):
    content_list.append(span.string)
    json__var = "{" + "span." + str(json_var_name) + str(i) + "}"
    # this line makes it different to modify_string
    span.string = json__var
    variables_list.append(json__var)
    i += 1
    return content_list, variables_list, i

###############################################################
# Helper functions for Main Function 2: component_to_json 
###############################################################

# Extracts the string between curly brackets {string}
def extract_between_brackets(s):
    if s.startswith("{") and s.endswith("}"):
        return s[1:-1]
    
# h2.h2_0 (I remove the h2 at the front)   
def remove_front_tag(s):
    s = s.split('.', 1)[-1] if '.' in s else s
    return s

# Put the string in between quotation marks
def put_in_quotes(s):
    s = "\"" + str(s)+ "\""
    return s

# Puts the string into curly brackets
def put_in_brackets(s, comma_b):
    if comma_b: 
        s = "{ \n" + str(s)+ "    },\n"
    else: 
        s = "{ \n" + str(s)+ "    }\n"
    return s

# Puts it into JSON format of field_name:content
def field_formatted(field_name, content, comma):
    field_name = put_in_quotes(field_name)
    content = put_in_quotes(content)
    if comma: 
        text_formatted = f"    {field_name}: {content}, \n" 
    else: 
        text_formatted = f"    {field_name}: {content} \n"
    return text_formatted

"""
Puts it in the format: 

field_name: {
    child_name: content
}

"""
def parent_field_formatted_list(parent_field, child_field_list, content_list, comma_b):
    fields = put_in_quotes(parent_field)
    content = ""
    
    i = 0
    last_element = len(child_field_list) - 1
    for item in child_field_list:
        content_list_current = content_list[i]
        if content_list_current: 
            content_list_current = content_list_current.strip()
        if i == last_element: 
            comma = False
        else:
            comma = True
        item = remove_front_tag(item)
        child_text_formatted  = field_formatted(item, content_list_current, comma)
        content += child_text_formatted
        i += 1
    content_brackets = put_in_brackets(content, comma_b)
    all_text_formatted = f" {fields}: {content_brackets}"
    return all_text_formatted

###############################################################
# Helper functions for Main Function 3: move_css_to_file
###############################################################
def define_css_filenames(stitch_name, css_file):
    css_filename = f"{stitch_name}/{stitch_name}.css"
    css_filename_write = f"{stitch_name}/{stitch_name}_modified/{css_file}.css"
    return css_filename, css_filename_write

In [222]:
"""
Main Functions
"""

###############################################################
# Main Function 1: component_to_svelte
###############################################################

def component_to_svelte(stitch_name,svelte_filename, tag_list, attributes_list, str_bool_list, html_filename):
    
    # Read HTML File
    html_content = read_file(html_filename)

    # Turn HTML Content into a SOUP object
    soup = BeautifulSoup(html_content, 'html.parser')
    
    class HTMLModifier: 
        def __init__(self, soup):
            self.soup = soup
            self.variables_list_master = []
            self.content_list_master = []

        def get_modified_html(self, stitch_name, tag_list):
            variable_str = ""
            variable_str = twoD_list_to_string(variable_str, tag_list)
            svelte_file_content = write_hardcoded_content(variable_str, stitch_name, soup)
            
            return svelte_file_content, self.variables_list_master, self.content_list_master
        
##############################################################################################################################
# Main Function
##############################################################################################################################
##############################################################################################################################
        def modify_all_tags(self, soup, tag_list, attributes_list, str_bool_list):
            print(f"tag_list: {tag_list}")
            tag_i = 0
            for tag in tag_list:
                # How do you know if the tag has nested tags or not
                print(f"tag: {tag}")
                current_tag_object = soup.find_all(tag)
                print(f"current_tag_object: {current_tag_object}")
                len_tag_list = len(current_tag_object)
                tag_i, content_list, variable_list, soup = modify_tags(tag, tag_i, attributes_list, str_bool_list, soup)
                
#                 # It's only tag 
#                 if len_tag_list <= 1:
#                     tag_i, content_list, variable_list, soup = modify_tags(tag, tag_i, attributes_list, str_bool_list, soup)
                
#                 # Otherwise it has nested tags within it 
#                 else:
#                     print(f"The List is Longer than 1: {len_tag_list}")
#                     tag_i, content_list, variable_list, soup = self.modify_nested_tags(tag, tag_i, attributes_list, str_bool_list, soup)
                    
                self.content_list_master.append(content_list)
                self.variables_list_master.append(variable_list)
        
        # Delete when I can
        def modify_label_tags(self,soup, tag_list):
            all_label= soup.find_all('label')
            i = 0
            variables_list = []
            content_list = []
            
            for label in all_label:
                content_list.append(label.contents[0])
                label_var = "{label.label" + str(i) + "}"

                variables_list.append(label_var)
                label, extracted_tag = extract_tag(label)
                
                label.string = label_var
                label.append(extracted_tag)
                i += 1
                
            self.content_list_master.append(content_list)
            self.variables_list_master.append(variables_list)
            tag_list.append('label')
            return tag_list

#############################################################################################################################################################################################
##############################################################################################################################

        def modify_span_tags(self, soup, tag_list):
            all_span = soup.find_all('span')
            
            """
            # subtitle = [subtitle_i, 'subtitle', topper']
            # title, color, heading, header
            """
            span_classes = [[0, 'subtitle', 'topper'], [0, 'title', 'title'], [0, 'color', 'color'], [0, 'heading', 'heading'], [0, 'header', 'header']]   

            variables_list = []
            content_list = []
            
            for span in all_span: 
                str_span = str(span)
                for span_class in span_classes:
                    if span_class[2] in str_span: 
                        content_list, variables_list, span_class[0] = modify_span(span, span_class[1], span_class[0], content_list, variables_list)

            self.variables_list_master.append(variables_list)
            self.content_list_master.append(content_list)
            tag_list.append('span')
            return tag_list

    """
    Final Content to put in svelte file
    """

    modifier = HTMLModifier(soup)

    """
    Modify the tags content
    """

    modifier.modify_all_tags(soup, tag_list, attributes_list, str_bool_list)
    tag_list = modifier.modify_span_tags(soup, tag_list)
    tag_list = modifier.modify_label_tags(soup, tag_list)

    svelte_file_content, variables_list, content_list = modifier.get_modified_html(stitch_name, tag_list)
    write_files(svelte_filename , svelte_file_content) 
        
    return variables_list, content_list, tag_list
        
###############################################################
# Main Function 2: component_to_json 
###############################################################
def component_to_json(stitch_name, filename, variables_list, content_list, tag_list): 
    bracket_start = "{"
    bracket_end = "}"
    comma = True
    name = field_formatted("name", stitch_name, comma)
    fields = put_in_quotes("fields")
    content = ""
    i = 0
    child_list_var_per_tag = []
    child_list_cont_per_tag = []
    for variable in variables_list:
        j = 0
        child_list_var = []
        child_list_cont = []
        for item in variable:       
            # If it's a list flatten it out 
            if isinstance(item, list):
                k = 0
                for attribute in item: 
                    extracted_item = extract_between_brackets(attribute)
                    child_list_var.append(extracted_item)
                    child_list_cont.append(content_list[i][j][k])
                    k += 1
            # If it's just a string then add it 
            else:
                extracted_item = extract_between_brackets(item)
                child_list_var.append(extracted_item)

                child_list_cont.append(content_list[i][j])
            j += 1
        child_list_var_per_tag.append(child_list_var)
        child_list_cont_per_tag.append(child_list_cont)
        i += 1

    i = 0
    last_element = len(tag_list) - 1
    for parent in tag_list: 
        if i == last_element:
            comma_b = False
        else: 
            comma_b = True
        text_formatted = parent_field_formatted_list(parent, child_list_var_per_tag[i], child_list_cont_per_tag[i], comma_b)
        content += text_formatted
        i += 1
        
    JSON_str = f"{bracket_start}\n {name} {fields}: {bracket_start}\n{content}{bracket_end}\n{bracket_end}"
    file_content = JSON_str
    write_files(filename , file_content)
    
###############################################################
# Main Function 3: move_css_to_file
###############################################################

In [223]:
import os
from bs4 import BeautifulSoup
from collections import Iterable

stitch_name = "contact_486"
svelte_file = f"{stitch_name}_v2"

tag_list = ['p', 'h1', 'h2', 'h3', 'a', 'input', 'option', 'img', 'button', 'label', 'svg']
attributes_list = [None, None, None, None, ['href'], ['name', 'placeholder', 'type'], ['value'], ['src', 'alt', 'height', 'width'], ['type'], None, None]
str_bool_list = [True, True, True, True, True, False, True, False, True, True, False]

###############################################################
# Component to svelte
###############################################################
html_filename, svelte_filename = define_filenames(stitch_name, svelte_file)
variables_list, content_list, tag_list = component_to_svelte(stitch_name, svelte_filename, tag_list, attributes_list, str_bool_list, html_filename)

###############################################################
# Component to JSON
###############################################################
json_filename = f"{stitch_name}/{stitch_name}_modified/{stitch_name}.json"
component_to_json(stitch_name, json_filename, variables_list, content_list, tag_list)

###############################################################
# Component to css file
###############################################################
css_file = "global.css"
css_filename, css_filename_write = define_css_filenames(stitch_name, css_file)

css_content = read_file(css_filename)
# either global.css, local.css or critical.css
write_files(css_filename_write, css_content)

tag_list: ['p', 'h1', 'h2', 'h3', 'a', 'input', 'option', 'img', 'button', 'label', 'svg']
tag: p
current_tag_object: [<p class="cs-text">
                We would love to hear about your project and discuss how we can help. Please fill the form on the right side to schedule a call or discussion and meeting on the your convenient time.
            </p>]
tag: p
 attributes_list[tag_i]: None 
 str_bool: True
 tag_i: 0 

len(current_tag): 1


tag: h1
current_tag_object: []
tag: h2
current_tag_object: [<h2 class="cs-title">Talk to us about your <span class="cs-color">garden plan!</span></h2>]
tag: h2
 attributes_list[tag_i]: None 
 str_bool: True
 tag_i: 2 

len(current_tag): 2


length longer than 1
ENTEREEDDD
type: <class 'str'>
i: 0, json_var: {h2.h20}
current_tag: <h2 class="cs-title">Talk to us about your <span class="cs-color">garden plan!</span></h2>
extracted_tag: [<span class="cs-color">garden plan!</span>]
new current_tag: <h2 class="cs-title">{h2.h20}</h2>, <class 'bs4.element.T

AttributeError: 'NoneType' object has no attribute 'append'

### codestitch_to_working_website
https://www.brentwoodstarplumbing.com/

Go to terminal and write "tree" and it generates the tree of the folders

1. [x] Take the stitches from the plumbing website and put them into a clean format
2. Go through each stitch and convert it - make sure I check the svelte, json and css code for each stitch
3. Push the github to my laptop 
4. Make a new website on my laptop 
5. Deploy it and check it works

-------
1. [x] Take a filepath as input
2. [x] rethink the folder structure
3. Change the conditionals for buttons
4. Add the svg tag

In [63]:
import os
from bs4 import BeautifulSoup
from collections import Iterable

# Global Components
navigation = 'cs-navigation-1343'
footer = 'cs-footer-710'
banner = 'banner-599'

# Pages
page_1 = 'Home'

# Page 1 Components 
hero = 'hero-408'
services = 'services-448'
why_choose_us = 'process-687'
additional_services = 'services-368'
featured_service_1 = 'RPsbs-1163'
reviews = 'reviews-64'
cta = 'cta-1072'

global_components_list = [navigation, footer, banner]
critical_components_list = [hero]
local_components = [services, why_choose_us, additional_services, featured_service_1, reviews, cta]

In [70]:
def define_filenames(folder_path, stitch_name, svelte_file):
    html_filename = f"{folder_path}{stitch_name}/{stitch_name}.html"
    svelte_filename = f"{folder_path}{stitch_name}/{stitch_name}_modified/{svelte_file}.svelte"    
    return html_filename, svelte_filename

def define_css_filenames(folder_path, stitch_name, css_file):
    css_filename = f"{folder_path}{stitch_name}/{stitch_name}.css"
    css_filename_write = f"{folder_path}{css_file}.css"
    return css_filename, css_filename_write

#############################
# cs-navigation-1343
#############################

stitch_name = navigation
svelte_file = f"{stitch_name}"
css_file = "global"
folder_path = "plumbing_stitches/"
json_filename = f"{folder_path}{stitch_name}/{stitch_name}_modified/{stitch_name}.json"

tag_list = ['p', 'h1', 'h2', 'h3', 'a', 'input', 'option', 'img', 'button']
attributes_list = [None, None, None, None, ['href'], ['name', 'placeholder', 'type'], ['value'], ['src', 'alt', 'height', 'width'], None]
str_bool_list = [True, True, True, True, True, False, True, False, True]

# Component to svelte
# Define HTML and svelte filename
html_filename, svelte_filename = define_filenames(folder_path, stitch_name, svelte_file)
variables_list, content_list, tag_list = component_to_svelte(stitch_name, svelte_filename, tag_list, attributes_list, str_bool_list, html_filename)

# Component to JSON
component_to_json(stitch_name, json_filename, variables_list, content_list, tag_list)

# Component to CSS
css_filename, css_filename_write = define_css_filenames(folder_path,stitch_name, css_file)
css_content = read_file(css_filename)
write_files(css_filename_write, css_content)